<a href="https://colab.research.google.com/github/romanfen/F21DL-G6/blob/main/Lab11_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up and Imports

In [1]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Import Smile Data

In [26]:
data = np.load('smiley_X.npy', allow_pickle=True)

OSError: ignored